In [4]:
from src import utils
import numpy as np
import pandas as pd
import dill
from importlib import reload

In [ ]:
reload(utils)
df = utils.Extract_and_Save_from_PDB(input_file='/scratch-scc/users/u14286/piplines/Bind/data/large/alphafold_db/test_data/AF-P11388-F1-model_v4.pdb',
                                 from_dill=False,
                                saving_dir='tmp/', k_nearest=5, inteacting_residues=False,
                              un_dn=False, outtype='dill', save_file=False)

print(df)

#### 1- Extract for id AF-P11388-F1-model_v4
PDB State AF-P11388-F1-model_v4
#### 3- Structure cleaned and renumbered AF-P11388-F1-model_v4


sphere_mange_arete: inconcistence


Append time: 4.76837158203125e-07 Iter time: 0.537541389465332 Before centroid calc: 0.5374197959899902 SASA time: 4.0531158447265625e-06


In [2]:
from src import utils, constants
import numpy as np
import pandas as pd
import dill
with open('/scratch-scc/users/u14286/piplines/Bind/tmp/AF-Q9VT52-F1-model_v4.dill', 'rb') as f:
    data = dill.load(f)

In [ ]:



tfmanager = utils.TFRecordManager(tfrecord_path='tmp/test.tfrecord', feature_dim=86)
tfmanager.write_samples(features=arr, labels=labels_arr, ID_arrs=IDs)
train_ds = tfmanager.read_dataset()

In [ ]:

from src import utils
import numpy as np
import pandas as pd
import dill
from importlib import reload
import time
reload(utils)

s = time.time()
output = utils.Extract_and_Save_from_PDB(input_file='/scratch-scc/users/u14286/piplines/Bind/data/large/alphafold_db/test_data/AF-Q9VT52-F1-model_v4.pdb',
                                 from_dill=False,
                                saving_dir='tmp/', k_nearest=5, inteacting_residues=False,
                              un_dn=False, outtype='tfrecord', save_file=False)

print(time.time() - s)
features, labels_arr, IDs, columns, plddt_arr = output


#### 1- Extract for id AF-Q9VT52-F1-model_v4
PDB State AF-Q9VT52-F1-model_v4
#### 3- Structure cleaned and renumbered AF-Q9VT52-F1-model_v4
Append time: 2.384185791015625e-07 Iter time: 0.0010123252868652344 Before centroid calc: 0.0009169578552246094 SASA time: 2.1457672119140625e-06
#### 4- pdb to df done AF-Q9VT52-F1-model_v4
################SIDE (120, 3)


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1552e5471f10>>
Traceback (most recent call last):
  File "/user/a.hajialiasgarynaj01/u14286/miniforge3/envs/Bind/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
tfmanager = utils.TFRecordManager(tfrecord_path='tmp/test2.tfrecord', feature_dim=86)
tfmanager.write_samples(features=features, labels=labels_arr, ID_arrs=IDs, plddt=plddt_arr)
train_ds = tfmanager.read_dataset()